# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

Question 1:  What test is appropriate?  Does CLT apply?

No, the data (both independent [apparent race] and dependent [callback] variables) are binary, not normally distributed, so CLT doesn't apply.

We need a correlation test appropriate for binary variables.  I think the most appropriate version is known as the Phi test, which is basically a chi squared test (and the equivalent of a Pearson test for binary quantities).  Apparently sklearn has a version called matthews_corrcoef (this is the first time I've looked for it in python). The documentation for pearson says that it assumes normalized distributions, so the phi test is what we need.

BUT I actually ran it both ways below, and I got the same correlation coefficient using both pearsonr and matthews_corrcoef out to 4 sigfigs, so it looks like it works either way.  And the built-in pearsonr provides a p-value (rather than just a correlation coefficient), so I'm happy to use that.

There's a chance that the unstated intent here is for me to use the bootstrapping method described in the DataCamp course and use the p-value as the test statistic, but since I'm not sure whether that's asked for here, and it hardly seems necessary, I'll start by not doing it.

Question 2: What are the null and alternate hypotheses?

The null hypothesis is that the preceived race of the applicant does not affect whether they get called by the emplyer.
The alternative hypothesis is that the perceived race of the applicant does affect whether they get called by the employer.  The direction of the correlation, if it exists, is most likely to indicate that a black-sounding name is less likely to be called back.


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import matthews_corrcoef

In [3]:
data = pd.io.stata.read_stata('data_race/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

First, look at the call data and find the mean/std.  We'll need that to answer question 3. 

In [5]:
data.call.describe()

count    4870.000000
mean        0.080493
std         0.272079
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

Okay, mean is 0.08 and std is .272.  So most people don't get called, as we would expect.  

I was asked to calculate confidence interval and margin of error. Since it isn't specified, I'll use a 2-sigma confidence interval.


In [7]:
MarginOfError=2*data.call.std()/(data.call.count())**(1/2)
ConfidenceInterval=[data.call.mean()-MarginOfError, data.call.mean()+MarginOfError]
print(MarginOfError)
print((ConfidenceInterval))

0.00779760509575
[0.07269521176969175, 0.088290421961196616]


Next I'll find the p-value.  The pearsonr method works for this.

In [8]:
raceBinary=data.race=='b'
#matthews_corrcoef(data.call, raceBinary)
stats.pearsonr(data.call,raceBinary)

(-0.05887209917662773, 3.9408030032935854e-05)

Question 4:  The p-value is small, indicating that the null hypothesis is unlikely, and the correlation is negative, meaning that a black-sounding name for a candidate results in a lower probability of getting called.  All the numbers are small here, but he p-value is small and the correlation coefficient significant.  It does seem like there's a correlation between having a black-sounding name and NOT getting a call.

Question 5:  Absolutely not, I have not explored the data enough to indicate that this is the most important factor.  There are other variables that need to be controlled for.  It is likely that job experience and other factors are significant.  It is possible that, if race is correlated with one of the other factors, we will not be able to even sort out any causality (was it the race that resulted in the no-call, or another factor), but the correlation between race and calls is established.